In [50]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
import math
warnings.filterwarnings('ignore')

## Community

In [51]:
hfcoccps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfsoccps14.csv", sep=",", index_col='ID_S0').sort_index()
hfchdxps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfshdxps14.csv", sep=",", index_col='ID_S0').sort_index()
hfchfaps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfshfaps14.csv", sep=",", index_col='ID_S0').sort_index()
hfcchips16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfschips14.csv", sep=",", index_col='ID_S0').sort_index()
hfccfdps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfscfdps14.csv", sep=",", index_col='ID_S0').sort_index()
hfccelps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/comm/hfscelps14.csv", sep=",", index_col='ID_S0').sort_index()


In [52]:
#find missing indexes and add them to the others
msng = list(set(hfcoccps16.index) - set(hfchfaps16.index))
for m in msng:
    hfchfaps16 = hfchfaps16.append(pd.Series(name=m))

hfchfaps16 = hfchfaps16.sort_index()

msng = list(set(hfcoccps16.index) - set(hfchdxps16.index))
for m in msng:
    hfchdxps16 = hfchdxps16.append(pd.Series(name=m))

hfchdxps16 = hfchdxps16.sort_index()

msng = list(set(hfcoccps16.index) - set(hfccelps16.index))
for m in msng:
    hfccelps16 = hfccelps16.append(pd.Series(name=m))

hfccelps16 = hfccelps16.sort_index()

In [53]:
cohort = pd.DataFrame()
cohort['ID'] = hfcoccps16.index
cohort = cohort.set_index('ID')

cohort['Age'] = hfcoccps16['AGE'].tolist()

lst = []
for idx in hfcoccps16.index:
    pt = hfcoccps16.loc[idx]
    if pt['SEX'] == 'M': 
        lst.append(1)
    else:
        lst.append(2)
cohort['Gender'] = lst

#weight, convert to all be Kg
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA20A1'] == 'L': #convert lbs to Kg
        lst.append(pt['HFAA20A'] * 0.45359237)
    else:
        lst.append(pt['HFAA20A'])
        
cohort['Wt_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA20B1'] == 'L': #convert lbs to Kg
        lst.append(pt['HFAA20B'] * 0.45359237)
    else:
        lst.append(pt['HFAA20B'])

cohort['Wt_D'] = lst

cohort['BMI_D'] = hfcoccps16['BMI'].tolist()
cohort['EjF'] = hfcoccps16['LVEF_CUR'].tolist()

#NYHA
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA24D'] == 'N': 
        lst.append(0)
    else:
        lst.append(pt['HFAA24D'])
cohort['NYHA'] = lst

cohort['TotalHospDays'] = hfcoccps16['LOS'].tolist()

cohort.head(5)

,Age,Gender,Wt_B,Wt_D,BMI_D,EjF,NYHA,TotalHospDays
ID,,,,,,,,
S106083,66,2,86.000000,88.500000,36.865164,NaN,0,10
S106611,67,1,80.200000,75.000000,24.417188,30.0,4,5
S106644,76,2,57.200000,57.900000,23.346821,NaN,0,5
S106724,78,2,84.640336,81.646627,29.113507,NaN,0,2
S107351,70,1,59.874193,NaN,20.112497,40.0,0,29


In [ ]:
#conditions, Ischemic
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA6A'] == 'Y' or pt['HFAA6A'] == 'Y' or pt['HFAA11K'] == 'Y' or  pt['HFAA16J']== 'Y' or  pt['HFAA76']== 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ISCH'] = lst

#Nonischemic
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA6B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['NonISCH'] = lst

#AF
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B1'] == 'Y' or pt['HFAA16K'] == 'Y' or pt['HFAA26C'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['AF'] = lst

#AlchE
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA9B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['AlchE'] = lst

#Angina
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11A'] == 'Y' or pt['HFAA16J'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ANGP'] = lst

#ARRH
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B1'] == 'Y' or pt['HFAA11B2'] == 'Y' or pt['HFAA11B3'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ARRH'] = lst

#CARREST
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11D'] == 'Y' or pt['HFAA74'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['CARREST'] = lst

#COPD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA10B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['COPD'] = lst

#Depr
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA14B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DEPR'] = lst

#Diab
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA12A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIAB'] = lst

#HEPT
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA22D'] == 'Y' or pt['HFAA22C'] == 'Y' or pt['HFAA12C'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['HEPT'] = lst

#HTN
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11J'] == 'Y' or pt['HFAA11L'] == 'Y' or pt['HFAA29D10'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['HTN'] = lst

#Malig
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA9E'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['MALIG'] = lst

#Malig
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA13A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['RENAL'] = lst

#Smok
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA9H'] == 'Y' or pt['HFAA9G'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['SMOKING'] = lst

#Stroke TIA
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA14A'] == 'Y' or pt['HFAA75'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['StrokeTIA'] = lst

#VAHD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11M'] == 'Y' or pt['HFAA28L'] == 'Y' or pt['HFAA28I'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VAHD'] = lst

#VF
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B3'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VF'] = lst

#VHD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11O'] == 'Y' or pt['HFAA29D4'] == 'Y' or pt['HFAA29D5'] == 'Y' or pt['HFAA29D6'] == 'Y' or pt['HFAA29D7'] == 'Y' or pt['HFAA29D8'] == 'Y' or pt['HFAA32B4'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VHD'] = lst

#VT
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B3'] == 'Y' or pt['HFAA26F'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VT'] = lst

#CABG
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E1'] == 'Y' or pt['HFAA51'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['CABG'] = lst

#HTRANS
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA56'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['HTRANS'] = lst

#ICD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E5'] == 'Y' or pt['HFAA11I'] == 'Y' or pt['HFAA47'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ICD'] = lst

#Pace
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E4'] == 'Y' or pt['HFAA49'] == 'Y' or pt['HFAA50'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['PACE'] = lst

#PTCI
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E2'] == 'Y' or pt['HFAA52'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['PTCI'] = lst


cohort.head(5)

In [146]:
# Labs
cohort['BUN_D'] = hfchfaps16['HFAA45B'].tolist()
cohort['CRT_B'] = hfchfaps16['HFAA44A1'].tolist()

#CRT
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if not np.isnan(pt['HFAA44B1']):
        lst.append(pt['HFAA44B1'])
    else:
        lst.append(pt['HFAA44B'])

cohort['CRT_D'] = lst
cohort['HEC_D'] = hfchfaps16['HFAA38B'].tolist()
cohort['HEM_D'] = hfchfaps16['HFAA37B'].tolist()
cohort['SOD_D'] = hfchfaps16['HFAA43B'].tolist()

#Meds
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA59'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ACE_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA59A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ACE_D'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA65'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['BET_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA65A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['BET_D'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA71'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['NIT_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA71A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['NIT_D'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA68'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIUR_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA68A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIUR_D'] = lst

cohort

,Age,Gender,Race,Weight_B,Weight_D,BMI_D,EjF,NYHA,TotalHospDays,ISCH,...,HEM_D,SOD_D,ACE_B,ACE_D,BET_B,BET_D,NIT_B,NIT_D,DIUR_B,DIUR_D
ID,,,,,,,,,,,,,,,,,,,,,
S105655,76,1,1,NaN,NaN,NaN,NaN,NaN,11.0,0,...,13.4,135.0,1,1,1,1,1,1,0,0
S105737,76,2,1,67.70,63.3,28.185923,40.0,NaN,3.0,0,...,11.0,138.0,0,0,1,1,0,0,0,0
S106083,66,2,1,86.00,88.5,36.865164,NaN,0,10.0,0,...,10.4,136.0,0,0,1,1,0,0,1,1
S106142,77,2,1,79.40,66.8,22.391913,60.0,0,11.0,0,...,12.3,140.0,0,0,1,1,0,0,1,1
S106248,66,2,1,91.00,93.3,37.621043,NaN,NaN,4.0,1,...,13.8,137.0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S214719,83,2,1,115.70,114.6,39.570139,NaN,NaN,6.0,1,...,12.6,140.0,0,0,0,0,1,0,1,0
S214738,73,2,1,52.30,52.0,22.020398,65.0,NaN,11.0,0,...,9.6,136.0,0,0,1,1,0,0,0,0
S214744,74,2,1,87.10,87.1,35.121038,NaN,NaN,3.0,0,...,14.4,141.0,0,1,1,1,1,1,1,1


In [147]:
cohort['HRTRT'] = hfchfaps16['HFAA18A'].tolist()
cohort['BPSYS_B'] = hfchfaps16['HFAA17A'].tolist()
cohort['BPDIAS_B'] = hfchfaps16['HFAA17B'].tolist()
cohort['BPSYS_D'] = hfchfaps16['HFAA17C'].tolist()
cohort['BPDIAS_D'] = hfchfaps16['HFAA17D'].tolist()

cohort['RAP'] = hfchfaps16['HFAA31B1'].tolist()
cohort['PAS'] = hfchfaps16['HFAA31B2'].tolist()
cohort['PCWP'] = hfchfaps16['HFAA31B3'].tolist()
cohort['CO'] = hfchfaps16['HFAA31B4'].tolist()
cohort['CI'] = hfchfaps16['HFAA31B5'].tolist()


idx = sorted(set(cohort.index))

#MAP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPSYS_B'] + ((2 * t['BPDIAS_B']) / 3)
    f = t['BPSYS_D'] + ((2 * t['BPDIAS_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['MAP_B'] = base
cohort['MAP_D'] = final

#CPI
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = (t['CI'] * t['MAP_B']) / 451
    f = (t['CI'] * t['MAP_D']) / 451
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['CPI_B'] = base
cohort['CPI_D'] = final
    
#PP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPSYS_B'] - t['BPDIAS_B']
    f = t['BPSYS_D'] - t['BPDIAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['PP_B'] = base
cohort['PP_D'] = final

#PPP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['BPSYS_B']
    f = t['PP_D'] / t['BPSYS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['PPP_B'] = base
cohort['PPP_D'] = final

#SVR
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = 80 * (t['MAP_B']  - t['RAP']) / t['CO']
    f = 80 * (t['MAP_D']  - t['RAP']) / t['CO']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['SVR_B'] = base
cohort['SVR_D'] = final

#RAT
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]
    b = t['RAP'] / t['PCWP']

    base.append(b)

cohort['RAT'] = base


#PPRatio
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['HRTRT']
    f = t['PP_D'] / t['HRTRT']
    c = f - b

    base.append(b)
    final.append(f)
    
cohort['PPRatio_B'] = base
cohort['PPRatio_D'] = final


#SAPi
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = (t['BPSYS_B'] - t['BPDIAS_B']) / t['PCWP']
    f = (t['BPSYS_D'] - t['BPDIAS_D']) / t['PCWP']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['SAPi_B'] = base
cohort['SAPi_D'] = final

#CPP
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['BPDIAS_B'] - t['PCWP']
    f = t['BPDIAS_D'] - t['PCWP']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['CPP_B'] = base
cohort['CPP_D'] = final

#PRAPRat
chg = []
base = []
final = []
for i in idx:
    t = cohort.loc[i]

    b = t['PP_B'] / t['RAP']
    f = t['PP_D'] / t['RAP']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
cohort['PRAPRat_B'] = base
cohort['PRAPRat_D'] = final


cohort.head(5)

,Age,Gender,Race,Weight_B,Weight_D,BMI_D,EjF,NYHA,TotalHospDays,ISCH,...,SVR_D,RAT,PPRatio_B,PPRatio_D,SAPi_B,SAPi_D,CPP_B,CPP_D,PRAPRat_B,PRAPRat_D
ID,,,,,,,,,,,,,,,,,,,,,
S105655,76,1,1,NaN,NaN,NaN,NaN,NaN,11.0,0,...,NaN,NaN,1.492958,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S105737,76,2,1,67.7,63.3,28.185923,40.0,NaN,3.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S106083,66,2,1,86.0,88.5,36.865164,NaN,0,10.0,0,...,NaN,NaN,0.734694,0.571429,NaN,NaN,NaN,NaN,NaN,NaN
S106142,77,2,1,79.4,66.8,22.391913,60.0,0,11.0,0,...,NaN,NaN,1.333333,1.333333,NaN,NaN,NaN,NaN,NaN,NaN
S106248,66,2,1,91.0,93.3,37.621043,NaN,NaN,4.0,1,...,NaN,NaN,0.531250,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
#Make separate datasets

#HEMO
varz = ['Age', 'Gender', 'Race', 'EjF', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 
                            'CI', 'MIXED', 'BPSYS', 'BPDIAS', 'HRTRT', 'MAP','MPAP', 'CPI',
                            'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat']

giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
        elif col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(df[col+"_D"])
        elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(0)
        elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(0)
            row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
    
    giantList.append(row0)
    giantList.append(row1)

varz.insert(0, 'ID')
hemoDF = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
hemoDF

not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found 

not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found 

not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found 

not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found 

not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found 

...

not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found 

not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found 

not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found 

not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found PAMN
not found MIXED
not found MPAP
not found PAPP
not found PAPi
not found PAD
not found 

,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
ID,,,,,,,,,,,,,,,,,,,,,
S105655,76,1,1,NaN,NaN,NaN,0,0,NaN,NaN,...,106.0,0.546392,0,NaN,NaN,1.492958,0,NaN,NaN,NaN
S105655,76,1,1,NaN,NaN,NaN,0,0,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
S105737,76,2,1,40.0,NaN,NaN,0,0,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
S105737,76,2,1,40.0,NaN,NaN,0,0,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
S106083,66,2,1,NaN,NaN,NaN,0,0,NaN,NaN,...,72.0,0.507042,0,NaN,NaN,0.734694,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S214744,74,2,1,NaN,NaN,NaN,0,0,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
S214752,80,2,1,60.0,NaN,NaN,0,0,NaN,NaN,...,120.0,0.521739,0,NaN,NaN,1.153846,0,NaN,NaN,NaN
S214752,80,2,1,60.0,NaN,NaN,0,0,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN


In [174]:
#save to file
hemoDF.to_csv("../Data Validation/ARIC/Original DataFrames/HemoAricCohort.csv")

In [175]:
#Make separate datasets

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']



giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
        elif col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(df[col+"_D"])
        elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(0)
        elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(0)
            row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
    
    giantList.append(row0)
    giantList.append(row1)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
allData

,Age,Gender,Race,Wt,BMI,InitialHospDays,TotalHospDays,NYHA,MLHFS,AF,...,DIUR,EjF,BPDIAS,BPSYS,HR,PV,MAP,PP,PPP,PPRatio
ID,,,,,,,,,,,,,,,,,,,,,
S105655,76,1,1,0,0.000000,0,11.0,NaN,0,0,...,0,NaN,88.0,194.0,0,0,252.666667,106.0,0.546392,1.492958
S105655,76,1,1,0,NaN,0,11.0,NaN,0,0,...,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
S105737,76,2,1,0,0.000000,0,3.0,NaN,0,1,...,0,40.0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
S105737,76,2,1,0,28.185923,0,3.0,NaN,0,1,...,0,40.0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
S106083,66,2,1,0,0.000000,0,10.0,0,0,0,...,1,NaN,70.0,142.0,0,0,188.666667,72.0,0.507042,0.734694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S214744,74,2,1,0,35.121038,0,3.0,NaN,0,1,...,1,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
S214752,80,2,1,0,0.000000,0,2.0,NaN,0,0,...,0,60.0,110.0,230.0,0,0,303.333333,120.0,0.521739,1.153846
S214752,80,2,1,0,24.616617,0,2.0,NaN,0,0,...,0,60.0,NaN,NaN,0,0,NaN,NaN,NaN,NaN


In [176]:
#save to file
allData.to_csv("../Data Validation/ARIC/Original DataFrames/AllDataAricCohort.csv")

In [ ]:
#Make Labels
labels = pd.DataFrame()
labels['ID'] = hfcoccps16.index
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    pt1 = hfccelps16.loc[idx]
    if pt1['CELB06'] == 'Y' or pt1['CELB12'] == 'Y' or pt['HFAA57'] == 'Y' or pt['HFAA56'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst

labels

In [ ]:
labels.to_csv("../Data Validation/ARIC/Original DataFrames/LabelsAricCohort.csv")